# Example: OWSLib extension for ESGF compute API

This notebook demonstrates a prototype of an [ESGF API](https://github.com/ESGF/esgf-compute-api) cliet implementation based on [OWSLib](https://github.com/geopython/OWSLib). On the server side, we are using [PyWPS](https://pywps.org/) running a mock ESGF compute process called `Emu.subset`. This subsetting process uses `xarray` to subset input NetCDF files.

Please check the OWSLib code on GitHub for details:

* Code: https://github.com/bird-house/OWSLib/blob/esgfwps/owslib/esgfapi.py
* Tests: https://github.com/bird-house/OWSLib/blob/esgfwps/tests/test_esgfapi.py

You can compare this with notebook examples of the original ESGF compute interface: 

* https://github.com/ESGF/esgf-compute-api
* https://github.com/ESGF/esgf-compute-api/tree/master/examples

<div class="alert alert-block alert-warning">
<b>Disclaimer:</b>  This prototype is incomplete. It's meant to show how we can leverage the OGC-related code base to meet ESGF requirements and avoid maintaining code by ourselves. That being said, all implementations need improvements and could use additional eye balls: OWSLib, OWSLib/esgfapi, PyWPS and ESGF-API itself. 
</div>

We could also use PyWPS for WPS service definitions and build a seperate ESGF compute library for processing functionality. We can define an abstract PyWPS process class which can be used (subclassed) to define new ESGF-API processes.

See: 
* https://github.com/ESGF/esgf-compute-wps
* https://github.com/bird-house/emu/blob/esgfwps/emu/processes/wps_esgf_subset.py
* https://pywps.org/
* http://xarray.pydata.org/en/stable/dask.html


## WPS client OWSLib/esgfapi

In [ ]:
from owslib.wps import WebProcessingService

**ESGF Access Token**

**TODO**: Use headers in WPS requests to transport an `api_key`. `api_key` can be handled either in pywps or security middleware like [Twitcher](https://twitcher.readthedocs.io/en/latest/).

In [ ]:
# ESGF Access Token
api_key = 'TOKEN'

# use headers
headers = {'api_key': api_key}

### Get Capabilities

Here we are using a [mock ESGF process](https://github.com/bird-house/emu/blob/esgfwps/emu/processes/wps_esgf_subset.py) from the `Emu` test server, due to certificate issues with the ESGF compute demo service. 

In [ ]:
# client = WebProcessingService('https://aims2.llnl.gov/wps/', api_key=api_key, verify=False)
client = WebProcessingService('https://bovec.dkrz.de/ows/proxy/emu', headers=headers, verify=True)
# client = WebProcessingService('http://localhost:5000/wps', headers=headers, verify=True)

In [ ]:
for p in client.processes:
    print(p.identifier)

### Describe Process

In [ ]:
proc = client.describeprocess(
    'Emu.subset'  # 'CDAT.subset'
)
proc.identifier

In [ ]:
for inpt in proc.dataInputs:
    print(inpt.identifier, inpt.dataType)

### WPS Process Inputs

**Domain**

**TODO**: can we use WPS boundingbox to describe domain? Are there other OGC concepts we can use?

In [ ]:
from owslib.esgfapi import Domain, Dimension

In [ ]:
d0 = Domain([
    Dimension('time', 0, 1, crs='indices'),
    Dimension('lat', 40, 60, crs='values'),
    Dimension('lon', 0, 20, crs='values'),
])

In [ ]:
# show json
print(d0.json)

In [ ]:
# add domain to WPS inputs
inputs = [('domain', d0)]

**Variable**

In [ ]:
from owslib.esgfapi import Variable

**TODO**: Should we use the file transportation layer of PyWPS?

In [ ]:
# data files we want to process
files = [
    # OpenDAP, CORDEX EUR-44, tasmax, climate index SU (summer days)
    'http://opendap.knmi.nl/knmi/thredds/dodsC/CLIPC/gerics/climatesignalmaps/EUR-44/tasmax/su_python-2-7-6_GERICS_ens-multiModel-climatesignalmap-rcp85-EUR-44_yr_20700101-20991231_1971-2000.nc',
]


In [ ]:
# add them one by one to WPS inputs as Variable
for x in files:
    # variable=su (summer days climate index)
    inputs.append(('variable', Variable(uri=x, var_name='su')))  

In [ ]:
# show all WPS inputs
for inp in inputs:
    print(inp[1])

### Execute

In [ ]:
exec = client.execute(proc.identifier, inputs=inputs)

In [ ]:
exec.isComplete()

In [ ]:
exec.isSucceded()

**Outputs**

**TODO**: return multiple output files ... maybe using metalink.

See: https://github.com/bird-house/emu/issues/64

In [ ]:
# show the output ... well, in case of Emu WPS it is just a dummy.
for output in exec.processOutputs:
    print(output.identifier, output.reference)

**Plot Preview**

In [ ]:
from IPython.display import Image

In [ ]:
Image(exec.processOutputs[2].reference)

## Birdy


Birdy offers through its `WPSClient` class a simplified interface to WPS processes, but it does not yet support the CWT-API extension (see [issue 102](https://github.com/bird-house/birdy/issues/102)). 
Look at the [doc](https://birdy.readthedocs.io/en/latest/) for more info. 

In [ ]:
from birdy import WPSClient
wps = WPSClient('https://bovec.dkrz.de/ows/proxy/emu')
help(wps.emu_subset)

In [ ]:
resp = wps.emu_subset(variable=Variable(uri=files[0], var_name='su'))